This is intended to be a function file for various things called in the course of correlated noise documents. 
Hopefully, functions can go here and stop cluttering up other files. Like display files. Good luck to me.

If i get really ambitious, I can start creating class definitions, etc for stuff. We'll see how it goes.
First off: get basic functions in fast versions. Preferably, jitted versions.

In [96]:
import numpy as np
from numba import jit, prange
import math
import csv
import datetime
import os

In [ ]:
```
A one-run file with all the options. Options desired:
    - number of deciders (input)
    - preimposed stoptime (input)
    - tracks k
    - has options to split p, update size by corr vs indepdenent (input)
    - tracks number of steps
```

In [86]:

@jit
def oneRun(n, c, th, p, 
               update = 1, 
                numDeciders = 1,
                maxTime = 100, 
                pInd = 0, updateInd = 0):

    # set unset settings and 
    # reconfigure settings for use
#     n = numAgents
#     p = probPos
    q = p/np.exp(update)
    if pInd == 0:
        pInd = p
    if updateInd == 0:
        updateInd = update
        qInd = q
    else:
        qInd = pInd/np.exp(updateInd)
    
    time = 0
    k = 0
    acc = 1
    
    numUndecided = n         # numUndecided is the number of agents who have not yet decided;
                            # the 'number of undecided agents remaining in the clique'
        
    tilEnd = numDeciders # tilEnd will tell us how many deciders we still need;
                            # numDeciders will retain the total number desired if we hit 
                            # maxTime and need to reset


    xEvolve = np.zeros(n)
    orderedAcc = np.zeros(numDeciders)
    orderedK = np.zeros(numDeciders)
    orderedTime = np.zeros(numDeciders)
    orderedOver = np.full(numDeciders, False)
    
    while tilEnd > 0:
        
        # evolve evidence
        while(np.abs(xEvolve).max()<th) and (tilEnd > 0):
            time += 1
            
            # a check to keep us out of infinite loops and
            # interrupt us if we want to only accept responses at a certain time(stopTime)
            # causes a reset so we start over.
            if time == maxTime:
                time = 0
                k = 0 
                acc = 1
                numUndecided = n
                tilEnd = numDeciders
                xEvolve = np.zeros(n)
                break

            new = np.zeros(numUndecided)
            
            isCorr = np.random.uniform(0,1)
            if isCorr <= c:
                check = np.random.uniform(0, 1)
                if check <= p:
                    new = np.full(numUndecided, float(update))
                elif check > 1-q:
                    new = np.full(numUndecided, float(-update)) 
                    
            else:
                check = np.random.rand(numUndecided)
                new[np.where(check < pInd)] = updateInd
                new[np.where(check >1-qInd)] = -updateInd
                k += 1

            xEvolve = xEvolve + new         
    
        if time == -1: break
        else:
            # take the next decider
            # This is stupid for large n: It needs to be able to take batches. 

            decider = np.where(np.abs(xEvolve)>=th)[0]
            decision = xEvolve[decider]/th
            xEvolve = np.delete(xEvolve, decider)
            numDeciding = decider.size
            over = np.full(numDeciding, False)
            over[np.where(np.abs(decision) > 1)] = True
            decision[np.where(decision < 0)] = 0
            decision[np.where(decision > 1)] = 1
            
            if tilEnd < numDeciding:
                numDeciding = tilEnd
                decision = decision[:numDeciding]
                over = over[:numDeciding]
                
            orderedAcc[n-numUndecided:n-numUndecided + numDeciding] = decision
            orderedK[n-numUndecided:n-numUndecided + numDeciding] = k
            orderedTime[n-numUndecided:n-numUndecided + numDeciding] = time
            orderedOver[n-numUndecided:n-numUndecided + numDeciding] = over
            
            numUndecided -= numDeciding
            tilEnd -= numDeciding
        

    return orderedAcc, orderedK, orderedTime, orderedOver

# ```
# orderedAcc: list of accuracies (1/0 coding)
# orderedK: list of k (number of indepedent observations)
# orderedTime - list of times of decisions
# orderedOver - whether the decision occured with private belief at threshold or over threshold
# ```

In [88]:
# Testing oneRun
n = 5; c = .4; th = 3; p = .3
print(oneRun(n, c, th, p, update = 2, numDeciders = 3, updateInd = 2))
# oneRun(n, c, th, p, update, 
#                 numDeciders = 1,
#                 maxTime = 100, 
#                 pInd = 0, updateInd = 0)

(array([1., 1., 1.]), array([2., 5., 8.]), array([2., 5., 8.]), array([ True,  True,  True]))


In [ ]:
```
A batch wrapper to make batches of things and save them:
    - accepts an end-product filename
    - accepts a kwargs set to pass to the one-run function
    - two versions, one parallel and one not
    - for the parralel version, use prange from numba 
        - this will require you to understand virtual environments if it ever goes on the school computers
    - the not parallel version write as it goes
    - the parallel version can write in batches
    - both should have a reportTime = True/False to determine if we need time updates as we go along.
```

In [94]:
# First, the parallel version innards:
# calls - 
# oneRun(n, c, th, p, update, 
#                 numDeciders = 1,
#                 maxTime = 100, 
#                 pInd = 0, updateInd = 0)
# oneRun returns : orderedAcc, orderedK, orderedTime, orderedOver (all np.arrays 3 floats, 1 bool)

@jit 
def innerBatchParallel(numTrials, 
                     n, c, th, p, 
                     update = 1, 
                     numDeciders = 1,
                     maxTime = 100, 
                     pInd = 0, updateInd = 0):
        
    # make empty arrays (so we can make parallel thingies)
    # shapes will be (numTrials, numDeciders) so each trial will be a row
    accs = np.zeros((numTrials, numDeciders))
    ks = np.zeros((numTrials, numDeciders))
    times = np.zeros((numTrials, numDeciders))
    overs = np.full((numTrials, numDeciders), False)
    
    for i in prange(numTrials):
        acc, k, time, over = oneRun(n, c, th, p, update,
                                   numDeciders, maxTime, pInd, updateInd)
        accs[i,:] = acc
        ks[i,:] = k
        times[i,:] = time
        overs[i,:] = over
        
    return accs, ks, times, overs


In [99]:
# Then, its wrapper that actually saves the (raw) data (will have processing functions later)
# ```
# saved columns:
#     n - number of agents
#     c - probability of correlated observation
#     th - threshold
#     pCorr - probability of positive observation for correlated observation
#     pInd - probability of positive observation for indepdendent observation
#     upCorr - update size for correlated observation
#     upInd - update size for independent observation
#     acc- (0/1) coding for inaccurate/accurate
#     time- number of timesteps
#     k - number of indepdent observations
#     over - whether the private belief was over threshold at the time of decision
#             (if belief was at threshold, over = False)
#     order - order of decision. First decider = 1, second decider = 2, etc.
# ```

def runBatchParallel(dataFile,
                     numTrials, 
                     n, c, th, p, 
                     update = 1, 
                     numDeciders = 1,
                     maxTime = 100, 
                     pInd = 0, updateInd = 0):

    # create data
    acc, k, time, over = innerBatchParallel(numTrials, 
                 n, c, th, p, 
                 update, 
                 numDeciders,
                 maxTime, 
                 pInd, updateInd)
    
    # fill in blank values, if they are blank
    if pInd == 0: pInd = p
    if updateInd == 0: updateInd = update
    
    # if the file doesn't exist, we want to make sure that we have our 
    # column names added to the first row
    firstTime = False
    if not os.path.exists(dataFile):
        print("New file!")
        firstTime = True

    # save data
    with open(dataFile, 'a+') as file:
        f = csv.writer(file, dialect='excel')
        if firstTime:
            f.writerow(["n", "c", "th",
                        "pCorr", "pInd", "upCorr", "upInd",
                        "acc","time", "k", "over", "order"])

        for i in range(numTrials):
            for j in range(numDeciders):
                f.writerow([n, c, th,
                            p, pInd, update, updateInd,
                            acc[i,j],
                            k[i,j],
                            time[i,j], 
                            over[i,j],
                            j+1 ])

In [100]:
# Testing runBatchParallel
n = 5; c = .4; th = 3; p = .3; numTrials = 6
print(runBatchParallel('testFile.csv',
                       numTrials, n, c, th, p, update = 1, 
                       numDeciders = 3, pInd = .7, updateInd = 2))

New file!
None


In [ ]:
```
An outer wrapper to make iterative batches of things:
    - accepts lists of n, pc, p1c, multc, mult1c <- update size, th, stoptime etc to iterate over
    - has stable or changing numTrials? 
    - accepts end-product filename to pass to batch wrapper
    - creates a kwargs set to pass to the batch wrapper, and thence to one-run function
```

In [152]:
def runSwath(nList, thList, cList, pCorrList, updateCorrList, pIndList, updateIndList,
             numDeciders, maxTime,
            numTrials, saveFile):
    # make sure all lists are, in fact, lists
    # so that we may iterate over them, even if they are of length 1. Boo.
    shouldBeLists = [nList, thList, cList, pCorrList, updateCorrList, pIndList, updateIndList]
    
    for i in range(len(shouldBeLists)):
        item = shouldBeLists[i]
        if not (type(item) == list):
            if type(item) == tuple:
                #print("was tuple")
                item = list(item)
            else:
                #print("was single")
                item = [item]
        #print(type(item))
        shouldBeLists[i] = item
            
    #[print(type(i)) for i in shouldBeLists]
    nList, thList, cList, pCorrList, updateCorrList, pIndList, updateIndList = shouldBeLists
    # run the stuff that makes and saves files
    for n in nList:
        numDec = numDeciders[n]
        for th in thList:
            for c in cList:
                for p in pCorrList:
                    for pInd in pIndList:
                        for update in updateCorrList:
                            for updateInd in updateIndList:
                                runBatchParallel(saveFile,
                                                 numTrials, 
                                                 n, c, th, 
                                                 p, update, 
                                                 numDec,
                                                 maxTime, 
                                                 pInd, updateInd)
            

In [153]:
# Testing runSwath

nList = [5,10,50]; thList = 3; cList = [.3,.7]
pCorrList = [.3,.7]; updateCorrList = 1
pIndList = [.3, .7]; updateIndList = (1,2)
numDeciders = {5:3, 10:2, 50:1}
maxTime = 100
numTrials = 5
saveFile = "testing123.csv"
runSwath(nList, thList, cList, pCorrList, updateCorrList, pIndList, updateIndList,
             numDeciders, maxTime,
            numTrials, saveFile)

3
<class 'int'>
New file!
2
<class 'int'>
1
<class 'int'>
